In [4]:
# Construction of dataset

import os, itertools, time
import subprocess
from xml.dom import minidom
from collections import Counter, OrderedDict
from operator import itemgetter
from nltk.corpus import wordnet
import tensorflow as tf
import tensorflow_hub as hub
from scipy import spatial
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.feature_extraction.text import TfidfVectorizer
import re
import numpy as np
import scipy.sparse as sp
import torch

USE_folder = "/home/vlead/USE"
alignment_folder = "reference-alignment/"

# Load reference alignments 
def load_alignments(folder):
    alignments = []
    for f in os.listdir(folder):
        doc = minidom.parse(folder + f)
        ls = list(zip(doc.getElementsByTagName('entity1'), doc.getElementsByTagName('entity2')))
        alignments.extend([(a.getAttribute('rdf:resource'), b.getAttribute('rdf:resource')) for (a,b) in ls])
    return alignments
        
reference_alignments = load_alignments(alignment_folder)
ontologies_in_alignment = [l.split(".")[0].split("-") for l in os.listdir("reference-alignment/")]

In [2]:
flatten = lambda l: [item for sublist in l for item in sublist]

class Ontology():
    def __init__(self, ontology):
        self.ontology = ontology
        self.ontology_obj = minidom.parse(ontology)
        self.root = self.ontology_obj.documentElement
        self.subclasses = self.parse_subclasses()
        self.object_properties = self.parse_object_properties()
        self.data_properties = self.parse_data_properties()
        self.triples = self.parse_triples()
        self.classes = self.parse_classes()
    
    def get_child_node(self, element, tag):
        return [e for e in element._get_childNodes() if type(e)==minidom.Element and e._get_tagName() == tag]
        
    def has_attribute_value(self, element, attribute, value):
        return True if element.getAttribute(attribute).split("#")[-1] == value else False
    
    def get_subclass_triples(self):
        return [(a,b,"subclass_of") for (a,b) in self.get_subclasses()]
    
    def parse_triples(self, union_flag=0, subclass_of=True):
        obj_props = self.object_properties
        data_props = self.data_properties
        props = obj_props + data_props
        all_triples = []
        for prop in props:
            domain_children = self.get_child_node(prop, "rdfs:domain")
            range_children = self.get_child_node(prop, "rdfs:range")
            domain_prop = self.filter_null([self.extract_ID(el) for el in domain_children])
            range_prop = self.filter_null([self.extract_ID(el) for el in range_children])
            if not domain_children or not range_children:
                continue
            if not domain_prop:
                domain_prop = self.filter_null([self.extract_ID(el) for el in domain_children[0].getElementsByTagName("owl:Class")])
            if not range_prop:
                range_prop = self.filter_null([self.extract_ID(el) for el in range_children[0].getElementsByTagName("owl:Class")])
            if domain_prop and range_prop:
                if union_flag == 0:
                    all_triples.extend([(el[0], el[1], self.extract_ID(prop)) for el in list(itertools.product(domain_prop, range_prop))])
                else:
                    all_triples.append(("###".join(domain_prop), "###".join(range_prop), self.extract_ID(prop)))
        if subclass_of:
            all_triples.extend(self.get_subclass_triples())
        return list(set(all_triples))
    
    def get_triples(self, union_flag=0, subclass_of=True):
        if union_flag == 0:
            return self.triples
        else:
            return self.parse_triples(union_flag = 1, subclass_of = False)

    def parse_subclasses(self, union_flag=0):
        subclasses = self.root.getElementsByTagName("rdfs:subClassOf")
        subclass_pairs = []
        for el in subclasses:
            inline_subclasses = self.extract_ID(el)
            if inline_subclasses:
                subclass_pairs.append((el, el.parentNode))
            else:
                level1_class = self.get_child_node(el, "owl:Class")
                if not level1_class:
                    continue
                if self.extract_ID(level1_class[0]):
                    subclass_pairs.append((level1_class[0], el.parentNode))
                else:
                    level2classes = level1_class[0].getElementsByTagName("owl:Class")
                    
                    subclass_pairs.extend([(elem, el.parentNode) for elem in level2classes if self.extract_ID(elem)])
        return subclass_pairs
        
    def get_subclasses(self):
        return [(self.extract_ID(a), self.extract_ID(b)) for (a,b) in self.subclasses]
    
    def filter_null(self, data):
        return [el for el in data if el]
    
    def extract_ID(self, element):
        element_id = element.getAttribute("rdf:ID") or element.getAttribute("rdf:resource") or element.getAttribute("rdf:about")
        return element_id.split("#")[-1]
    
    def parse_classes(self):
        class_elems = [self.extract_ID(el) for el in self.root.getElementsByTagName("owl:Class")]
        subclass_classes = list(set(flatten([el[:-1] for el in self.triples])))
        return list(set(self.filter_null(class_elems + subclass_classes)))
    
    def get_classes(self):
        return self.classes
    
    def get_entities(self):
        entities = [self.extract_ID(el) for el in self.root.getElementsByTagName("owl:Class")]
        return list(set(self.filter_null(entities)))

    def parse_data_properties(self):
        data_properties = [el for el in self.get_child_node(self.root, 'owl:DatatypeProperty')]
        fn_data_properties = [el for el in self.get_child_node(self.root, 'owl:FunctionalProperty') if el]
        fn_data_properties = [el for el in fn_data_properties if type(el)==minidom.Element and 
            [el for el in self.get_child_node(el, "rdf:type") if 
             self.has_attribute_value(el, "rdf:resource", "DatatypeProperty")]]
        inv_fn_data_properties = [el for el in self.get_child_node(self.root, 'owl:InverseFunctionalProperty') if el]
        inv_fn_data_properties = [el for el in inv_fn_data_properties if type(el)==minidom.Element and 
            [el for el in self.get_child_node(el, "rdf:type") if 
             self.has_attribute_value(el, "rdf:resource", "DatatypeProperty")]]
        return data_properties + fn_data_properties + inv_fn_data_properties
        
    def parse_object_properties(self):
        obj_properties = [el for el in self.get_child_node(self.root, 'owl:ObjectProperty')]
        fn_obj_properties = [el for el in self.get_child_node(self.root, 'owl:FunctionalProperty') if el]
        fn_obj_properties = [el for el in fn_obj_properties if type(el)==minidom.Element and 
            [el for el in self.get_child_node(el, "rdf:type") if 
             self.has_attribute_value(el, "rdf:resource", "ObjectProperty")]]
        inv_fn_obj_properties = [el for el in self.get_child_node(self.root, 'owl:InverseFunctionalProperty') if el]
        inv_fn_obj_properties = [el for el in inv_fn_obj_properties if type(el)==minidom.Element and 
            [el for el in self.get_child_node(el, "rdf:type") if 
             self.has_attribute_value(el, "rdf:resource", "ObjectProperty")]]
        return obj_properties + fn_obj_properties + inv_fn_obj_properties
    
    def get_object_properties(self):
        obj_props = [self.extract_ID(el) for el in self.object_properties]
        return list(set(self.filter_null(obj_props)))
    
    def get_data_properties(self):
        data_props = [self.extract_ID(el) for el in self.data_properties]
        return list(set(self.filter_null(data_props)))




In [5]:
# Extracting USE embeddings
def extractUSEEmbeddings(words):
    try:
        embed = hub.KerasLayer(USE_folder)
    except Exception as e:
        !mkdir $USE_folder
        !curl -L "https://tfhub.dev/google/universal-sentence-encoder-large/5?tf-hub-format=compressed" | tar -zxvC $USE_folder
        embed = hub.KerasLayer(USE_folder)
        pass
    word_embeddings = embed(words)
    return word_embeddings.numpy()

def cos_sim(a,b):
    return 1 - spatial.distance.cosine(a, b)

def camel_case_split(identifier):
    matches = re.finditer('.+?(?:(?<=[a-z])(?=[A-Z])|(?<=[A-Z])(?=[A-Z][a-z])|$)', identifier)
    return [m.group(0).lower() for m in matches]

def parse(word):
    return flatten([el.split("_") for el in camel_case_split(word)])
    

extracted_elems = []

for ont_name in list(set(flatten(ontologies_in_alignment))):
    ont = Ontology("conference_ontologies/" + ont_name + ".owl")
    entities = ont.get_entities()
    props = ont.get_object_properties() + ont.get_data_properties()
    triples = list(set(flatten(ont.get_triples(subclass_of=False))))
    extracted_elems.extend(entities + props + triples)

extracted_elems = list(set(extracted_elems))

entities_parsed = [parse(word) for word in extracted_elems]
inp = [" ".join(e) for e in entities_parsed]
vectorizer = TfidfVectorizer(token_pattern=r"(?u)\S+")
X = vectorizer.fit_transform(inp)
word2idx_tfidf = {word: i for (i, word)  in enumerate(vectorizer.get_feature_names())}
entity2idx_tfidf = {word: i for (i, word)  in enumerate(extracted_elems)}


extracted_elems.extend(flatten(entities_parsed))
extracted_elems = list(set(extracted_elems))

print ("Total number of extracted unique classes and properties from entire RA set: ", len(extracted_elems))

embeds = extractUSEEmbeddings([" ".join(parse(word)) for word in extracted_elems])
embeddings = dict(zip(extracted_elems, embeds))    

Total number of extracted unique classes and properties from entire RA set:  1228


In [108]:
# Feature generation

features_dict = {}

def get_tfidf_score(word, phrase):
    temp = X[entity2idx_tfidf[phrase]]
    return temp[:,word2idx_tfidf[word]][0,0]

def get_property_features(triple):
    weight = np.sum([get_tfidf_score(word, triple[-1]) for word in parse(triple[-1])])
    feats = []
    feats.append(weight*embeddings[triple[-1]]) # Property name
    feats.append([embeddings[el] for el in triple[0].split("###")]) # Domain 
    feats.append([embeddings[el] for el in triple[1].split("###")]) # Range
    return feats
    
def get_entity_features(entity):
    feats = []
    weight = np.sum([get_tfidf_score(word, entity) for word in parse(entity)])
    feats.append(weight * embeddings[entity]) # Entity name
#     feats.append(np.sum([get_tfidf_score(word, entity) * embeddings[word] for word in parse(entity)]))
    return feats
    
for ont_name in list(set(flatten(ontologies_in_alignment))):
    ont = Ontology("conference_ontologies/" + ont_name + ".owl")
    
    triples = ont.get_triples(union_flag=1, subclass_of=False)
    entities = ont.get_entities()
    props = ont.get_object_properties() + ont.get_data_properties()

    for triple in triples:
        features_dict[triple[-1]] = {"type": "triple", "features": get_property_features(triple)}
    for entity in entities:
        features_dict[entity] = {"type": "entity", "features": get_entity_features(entity)}
    for prop in props:
        if prop not in features_dict:
            features_dict[prop] = {"type": "property", "features": get_entity_features(prop)}


In [109]:
# Combinatorial mapping generation

all_mappings = []
for l in ontologies_in_alignment:
    ont1 = Ontology("conference_ontologies/" + l[0] + ".owl")
    ont2 = Ontology("conference_ontologies/" + l[1] + ".owl")
    
    ent1 = ont1.get_entities()
    ent2 = ont2.get_entities()
    
    obj1 = ont1.get_object_properties()
    obj2 = ont2.get_object_properties()
    
    data1 = ont1.get_data_properties()
    data2 = ont2.get_data_properties()
    
    mappings = list(itertools.product(ent1, ent2)) + list(itertools.product(obj1, obj2)) + list(itertools.product(data1, data2))
    
    all_mappings.extend([(l[0] + "#" + el[0], l[1] + "#" + el[1]) for el in mappings])
    

In [110]:
gt_mappings = [tuple([elem.split("/")[-1] for elem in el]) for el in reference_alignments]

def calc_sim_score(mapping):
    features = embedify(mapping)
    if features[0]["type"] != "triple" and features[1]["type"] != "triple":
        return cos_sim(*tuple([el["features"][0] for el in features]))
    elif features[0]["type"] == "triple"  and features[1]["type"] == "triple":
        sim_score = 0
        names, domains, ranges = [], [], []
        
        for elem in [el["features"] for el in features]:
            names.append(elem[0])
            domains.append(elem[1])
            ranges.append(elem[2])
        name_score = cos_sim(names[0], names[1])
#         domain_score = np.mean([cos_sim(*elem) for elem in itertools.product(domains[0], domains[1])])
#         ranges_score = np.mean([cos_sim(*elem) for elem in itertools.product(ranges[0], ranges[1])])
#         return ((name_score + domain_score + ranges_score)/(len(domains[0]) + len(ranges[0]) +  ))
#         return (name_score + domain_score + ranges_score)/3
        return name_score
    return cos_sim(*tuple([el["features"][0] for el in features])) 
        
        

def embedify(mapping):
    removed_hash = tuple([elem.split("#")[1] for elem in mapping])
    return (features_dict[removed_hash[0]], features_dict[removed_hash[1]])

data = {}
for mapping in all_mappings:
    if mapping in gt_mappings:
        data[(mapping[0], mapping[1])] = (calc_sim_score(mapping), "T")
    else:
        data[(mapping[0], mapping[1])] = (calc_sim_score(mapping), "F")


In [ ]:
def is_valid(test_onto, key):
    return tuple([el.split("#")[0] for el in key]) not in test_onto

data = OrderedDict(sorted(data.items(),  key=lambda x:x[1][0], reverse=True))
all_ont_pairs = list(set([tuple([el.split("#")[0] for el in l]) for l in data.keys()]))
results = []
failed = []
for i in list(range(0, len(all_ont_pairs), 3)):
    test_onto = all_ont_pairs[i:i+3]
    
    train_data = {elem: data[elem] for elem in data if tuple([el.split("#")[0] for el in elem]) not in test_onto}
    test_data = {elem: data[elem] for elem in data if tuple([el.split("#")[0] for el in elem]) in test_onto}

    opt_threshold, optimum_metrics = -1000, [-1000 for i in range(5)]
    t = time.time()
    for j,threshold in enumerate(np.arange(0.15, 1.0005, 0.01)):
        print ("threshold =", threshold, "Time = ", time.time()-t) 
        pred = []
        for i,key in enumerate(train_data):
            if train_data[key][0] > threshold:
                pred.append(key)

        tp = len([elem for elem in pred if train_data[elem][1] == "T"])
        fn = len([key for key in gt_mappings if key not in set(pred) and is_valid(test_onto, key)])
        fp = len([elem for elem in pred if train_data[elem][1] == "F"])

        try:
            precision = tp/(tp+fp)
            recall = tp/(tp+fn)
            f1score = 2 * precision * recall / (precision + recall)
            f2score = 5 * precision * recall / (4 * precision + recall)
            f0_5score = 1.25 * precision * recall / (0.25 * precision + recall)
        except Exception as e:
            print (e)
            continue
        print (precision, recall, f1score, f2score, f0_5score)

        if f1score > optimum_metrics[2]:
            optimum_metrics = [precision, recall, f1score, f2score, f0_5score]
            opt_threshold = threshold
    
    threshold = opt_threshold
    pred = []
    for i,key in enumerate(test_data):
        if test_data[key][0] > threshold:
            pred.append(key)

    curr = dict()
    
    curr["fn"] = [key for key in gt_mappings if key not in set(pred) and not is_valid(test_onto, key)]
    curr["fp"] = [elem for elem in pred if test_data[elem][1] == "F"]
    tp = len([elem for elem in pred if test_data[elem][1] == "T"])
    fn = len(curr["fn"])
    fp = len(curr["fp"])

    
    
    try:
        precision = tp/(tp+fp)
        recall = tp/(tp+fn)
        f1score = 2 * precision * recall / (precision + recall)
        f2score = 5 * precision * recall / (4 * precision + recall)
        f0_5score = 1.25 * precision * recall / (0.25 * precision + recall)
    except Exception as e:
        print (e)
        pass
            
    metrics = [precision, recall, f1score, f2score, f0_5score]
    failed.append(curr)
    results.append(metrics)

print ("Final Results:", np.mean(results, axis=0))

threshold = 0.15 Time =  4.267692565917969e-05
0.0038821550273907606 0.989010989010989 0.007733952049497294 0.019110714740731307 0.004847936395074497
threshold = 0.16 Time =  3.3263349533081055
0.004109088695440434 0.989010989010989 0.008184174231975873 0.020209580838323356 0.005131031337298798
threshold = 0.17 Time =  6.33664083480835
0.0043719740272357786 0.989010989010989 0.00870546509753345 0.0214800553708094 0.005458934658573964
threshold = 0.18000000000000002 Time =  9.057249307632446


In [79]:

def normalize_adj(mx):
    """Row-normalize sparse matrix"""
    rowsum = np.array(mx.sum(1))
    r_inv_sqrt = np.power(rowsum, -0.5).flatten()
    r_inv_sqrt[np.isinf(r_inv_sqrt)] = 0.
    r_mat_inv_sqrt = sp.diags(r_inv_sqrt)
    return mx.dot(r_mat_inv_sqrt).transpose().dot(r_mat_inv_sqrt)


def normalize_features(mx):
    """Row-normalize sparse matrix"""
    rowsum = np.array(mx.sum(1))
    r_inv = np.power(rowsum, -1).flatten()
    r_inv[np.isinf(r_inv)] = 0.
    r_mat_inv = sp.diags(r_inv)
    mx = r_mat_inv.dot(mx)
    return mx

def encode_onehot(labels):
    classes = set(labels)
    classes_dict = {c: np.identity(len(classes))[i, :] for i, c in enumerate(classes)}
    labels_onehot = np.array(list(map(classes_dict.get, labels)), dtype=np.int32)
    return labels_onehot

# idx_features_labels = np.genfromtxt("{}{}.content".format(path, dataset), dtype=np.dtype(str))
# features = sp.csr_matrix(idx_features_labels[:, 1:-1], dtype=np.float32)
# labels = encode_onehot(idx_features_labels[:, -1])

# build graph



idx = np.array(idx_features_labels[:, 0], dtype=np.int32)
idx_map = {j: i for i, j in enumerate(idx)}
edges_unordered = np.genfromtxt("{}{}.cites".format(path, dataset), dtype=np.int32)
edges = np.array(list(map(idx_map.get, edges_unordered.flatten())), dtype=np.int32).reshape(edges_unordered.shape)
adj = sp.coo_matrix((np.ones(edges.shape[0]), (edges[:, 0], edges[:, 1])), shape=(labels.shape[0], labels.shape[0]), dtype=np.float32)

# adj = adj + adj.T.multiply(adj.T > adj) - adj.multiply(adj.T > adj)

# features = normalize_features(features)
# adj = normalize_adj(adj + sp.eye(adj.shape[0]))

# idx_train = range(140)
# idx_val = range(200, 500)
# idx_test = range(500, 1500)

# adj = torch.FloatTensor(np.array(adj.todense()))
# features = torch.FloatTensor(np.array(features.todense()))
# labels = torch.LongTensor(np.where(labels)[1])

# idx_train = torch.LongTensor(idx_train)
# idx_val = torch.LongTensor(idx_val)
# idx_test = torch.LongTensor(idx_test)

Loading cora dataset...


In [ ]:
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F

class SiameseNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.name_embedding = nn.Embedding(len(embeddings), 512)
        self.name_embedding.load_state_dict({'weight': torch.from_numpy(np.array(embeddings))})
        
        self.dropout = dropout
        
        self.layer1 = nn.Linear(512, 240)
        self.layer2 = nn.Linear(240, 2)
        
    def forward(self, data):
        results = []
        for i in range(2):            
            x = F.dropout(data[i, self.dropout, training=self.training)
            x = torch.cat([att(x, adj) for att in self.attentions], dim=1)
            x = F.leaky_relu(F.dropout(x, self.dropout, training=self.training))
            results.append(x)
        

In [6]:
relations = []
for ont_name in list(set(flatten(ontologies_in_alignment))):
    ont = Ontology("conference_ontologies/" + ont_name + ".owl")
    
    triples = ont.get_triples()
    relations.extend([list(itertools.product(el[0].split("###"), el[1].split("###"))) for el in triples])
relations = list(set(flatten(relations)))    
entities = list(set(flatten(relations)))

In [105]:
entity_idx = {i for (i,elem) in enumerate(entities) }

In [106]:
[el for el in a if el not in list(set(entities_all))]

['dateTime',
 'string',
 'int',
 'date',
 'boolean',
 'anyURI',
 'positiveInteger',
 'unsignedLong']